In [118]:
import pandas as pd

In [124]:
from pymongo import MongoClient
import time
import api_keys
import requests
import gdax
import json

api = api_keys.api_keys()

client = MongoClient(api.mongo)
# specify the database and collection`
db = client.gdax.gdaxws
print('connected to mongo collection')

# and store in local variables
key = api.api_key
b64secret = api.secret_key
passphrase = api.passphrase

class GAC(gdax.AuthenticatedClient):
    def buy(self, **kwargs):
        kwargs["side"] = "buy"
        r = requests.post(self.url + '/orders',
                          data=json.dumps(kwargs),
                          auth=self.auth,
                          timeout=30)

        print(r)
        return r.json()

    def sell(self, **kwargs):
        kwargs["side"] = "sell"
        r = requests.post(self.url + '/orders',
                          data=json.dumps(kwargs),
                          auth=self.auth,
                          timeout=30)

        print('SOLD !!')
        print(r)
        db.orders.insert_one(r.json())
        return r.json()

    def getorders(self):
        """{'id': 'e22c9172-0276-47f7-b774-2559784c26aa', 'price': '999.85000000', 
            'size': '0.01000000', 'product_id': 'BTC-EUR', 'side': 'buy', 
            'type': 'limit', 'time_in_force': 'GTC', 'post_only': True,
            'created_at': '2018-08-17T14:57:18.551778Z', 'fill_fees': '0.0000000000000000',
            'filled_size': '0.00000000', 'executed_value': '0.0000000000000000', 
            'status': 'open', 'settled': False}"""
    
        self.openorders = client.get_orders()[0]
        self.df_openorders = pd.DataFrame(self.openorders)
        
    def getbalances(self):
        """{'id': '459d001f-0391-4e97-89e7-ae474275e2c9', 'currency': 'BTC',
            'balance': '0.0530287336346057', 'available': '0.0530287336346057',
            'hold': '0.0000000000000000', 'profile_id': '5100622b-3ed2-49e4-9810-c28fb96d30b3'} """

        self.balances = client.get_accounts()
        self.df_balances = pd.DataFrame(self.balances)

connected to mongo collection


In [125]:
client = GAC(key=key,b64secret=b64secret,
                 passphrase=passphrase,
                 api_url="https://api.pro.coinbase.com")
    
client.getbalances()

EUR balance at 164.2057972936818500
BTC balance at 0.0530287336346057


In [140]:
client.df_balances

,available,balance,currency,hold,id,profile_id
0,0.0530287336346057,0.0530287336346057,BTC,0.0000000000000000,459d001f-0391-4e97-89e7-ae474275e2c9,5100622b-3ed2-49e4-9810-c28fb96d30b3
1,0,0.0000000000000000,LTC,0.0000000000000000,ad1b7960-0dcb-440e-b60a-73ab56ab1035,5100622b-3ed2-49e4-9810-c28fb96d30b3
2,111.20729729368185,164.2057972936818500,EUR,52.9985000000000000,0a0fd378-71d0-4d5c-b7bb-0b48b1ff839c,5100622b-3ed2-49e4-9810-c28fb96d30b3
3,0,0.0000000000000000,ETH,0.0000000000000000,175a9d3f-e634-4575-bee7-96594bbaed3a,5100622b-3ed2-49e4-9810-c28fb96d30b3
4,0,0.0000000000000000,ETC,0.0000000000000000,a355123a-d206-4992-af06-20dce476611e,5100622b-3ed2-49e4-9810-c28fb96d30b3
5,0,0.0000000000000000,BCH,0.0000000000000000,5ea49af8-18ad-45d7-bf2f-e1f3b4bc7358,5100622b-3ed2-49e4-9810-c28fb96d30b3


In [167]:
balances = client.df_balances.query('currency in ("BTC","EUR")').T
balances.rename(columns=balances.loc['currency'], inplace=True)

In [170]:
balances.rename(columns=balances.loc['currency'], inplace=True)

In [154]:
bal

In [171]:
balances

,BTC,EUR
available,0.0530287336346057,111.20729729368185
balance,0.0530287336346057,164.2057972936818500
currency,BTC,EUR
hold,0.0000000000000000,52.9985000000000000
id,459d001f-0391-4e97-89e7-ae474275e2c9,0a0fd378-71d0-4d5c-b7bb-0b48b1ff839c
profile_id,5100622b-3ed2-49e4-9810-c28fb96d30b3,5100622b-3ed2-49e4-9810-c28fb96d30b3


In [351]:
from pymongo import MongoClient
import time
import api_keys
import requests
import gdax
import random as random
import json
import pandas as pd

api = api_keys.api_keys()

client = MongoClient(api.mongo)
# specify the database and collection`
db = client.gdax.gdaxws
print("connected to mongo collection")

# and store in local variables
key = api.api_key
b64secret = api.secret_key
passphrase = api.passphrase

class clearingmaster():
    def __init__(self, maxopenbuyorders = 4 ,**kwargs):
        self.maxopenbuyorders = maxopenbuyorders
        self.heartbeat_rate = 3600*1.5  # seconds before we auto cancel limit order
        self.marketBTCUSD = client.get_product_ticker("BTC-USD")
        self.marketBTCEUR = client.get_product_ticker("BTC-EUR")
        self.exchangerate = float(self.marketBTCEUR["price"]) / float(self.marketBTCUSD["price"])

    def getorders(self):
        """{'id': 'e22c9172-0276-47f7-b774-2559784c26aa', 'price': '999.85000000', 
            'size': '0.01000000', 'product_id': 'BTC-EUR', 'side': 'buy', 
            'type': 'limit', 'time_in_force': 'GTC', 'post_only': True,
            'created_at': '2018-08-17T14:57:18.551778Z', 'fill_fees': '0.0000000000000000',
            'filled_size': '0.00000000', 'executed_value': '0.0000000000000000', 
            'status': 'open', 'settled': False}"""
    
        openorders = client.get_orders()[0]
        df_openorders = pd.DataFrame(openorders).query('product_id == "BTC-EUR"')
        df_openorders['created_at'] = pd.to_datetime(df_openorders['created_at'])
        self.df_openorders = df_openorders
        #print('DAXY DEBUG', self.df_openorders)
        
    def getbalances(self):
        """{'id': '459d001f-0391-4e97-89e7-ae474275e2c9', 'currency': 'BTC',
            'balance': '0.0530287336346057', 'available': '0.0530287336346057',
            'hold': '0.0000000000000000', 'profile_id': '5100622b-3ed2-49e4-9810-c28fb96d30b3'} """

        balances = client.get_accounts()
        df_balances = pd.DataFrame(balances).query('currency in ("BTC","EUR")').T
        df_balances.rename(columns=df_balances.loc['currency'], inplace=True)
        
        l = ['available','balance','hold']        
        self.df_balances = client.cm_.df_balances.loc[l].astype('float')
        #print('DAXY DEBUG', self.df_balances)

    def getclearance(self, kwargs_dict):
        try:
            orderprice = kwargs_dict['price'] * kwargs_dict['size']
            print('DAXY CM order received total price: {}'.format(orderprice))
            self.getbalances()
            self.getorders()
        except KeyError:
            print('DAXY CM KeyError calculating orderprice')
            print('DAXY CM KeyError rejecting order request')
            return False

        if self.df_balances['EUR']['available'] <= orderprice:
            print('DAXY CM insufficient funds')
            return False
        elif len() >= self.maxopenbuyorders:
            print('DAXY CM maximum limit orders reached')
            return False
        else:
            return True

    def heartbeat(self, **kwargs):
        self.getorders()
        cutoffdate = pd.to_datetime(time.time()-self.heartbeat_rate, unit='s')
        to_terminate = self.df_openorders[self.df_openorders["created_at"]<cutoffdate]['id']
        for idtoterminate in to_terminate:
            response_cancel = client.cancel_order(idtoterminate) 
            print('DAXY CANCELLED old order : {}'.format(response_cancel))

class GAC(gdax.AuthenticatedClient):
    def __clearingmaster__(self):
        self.cm_ = clearingmaster()
        self.timeout = 15

    def buy(self, **kwargs):
        """client.buy(size="0.005000000",
                product_id="BTC-EUR",
                side="buy",
                stp="dc",
                type="limit")
                
            from CB PRO website
                {"id": "d0c5340b-6d6c-49d9-b567-48c4bfca13d2",
                "price": "0.10000000",
                "size": "0.01000000",
                "product_id": "BTC-USD",
                "side": "buy",
                "stp": "dc",
                "type": "limit",
                "time_in_force": "GTC",
                "post_only": false,
                "created_at": "2016-12-08T20:02:28.53864Z",
                "fill_fees": "0.0000000000000000",
                "filled_size": "0.00000000",
                "executed_value": "0.0000000000000000",
                "status": "pending",
                "settled": false}"""

        kwargs["side"] = "buy"
        kwargs["type"] = "limit"
        trade_request = self.cm_.getclearance(kwargs_dict=kwargs)
        
        if trade_request == True:
            r = requests.post(self.url + '/orders',
                            data=json.dumps(kwargs),
                            auth=self.auth,
                            timeout=30)

            print(r)
            r = r.json()
            r.update({'MONGOKEY':'BUY_ORDER','timestamp':time.time()})
            db.insert_one(r)
            print('DAXY BOUGHT inserted in mongo')
            return r
        else:
            print('DAXY CM no permission')

    def sell(self, **kwargs):
        """client.sell(size="0.005000000",
                product_id="BTC-EUR",
                side="sell",
                stp="dc",
                type="limit")"""

        kwargs["side"] = "sell"
        r = requests.post(self.url + '/orders',
                          data=json.dumps(kwargs),
                          auth=self.auth,
                          timeout=30)

        print('SOLD !!')
        print(r)
        r.update({'MONGOKEY':'SELL_ORDER','timestamp':time.time()})
        db.insert_one(r.json())
        print('DAXY SOLD inserted in mongo')
        return r.json()

if __name__ == "__main__":
    client = GAC(key=key,b64secret=b64secret,
                 passphrase=passphrase,
                 api_url="https://api.pro.coinbase.com")
    
    client.__clearingmaster__() # set clearing master
    client.cm_.getorders()
    p_change = {}
    counter=0

connected to mongo collection


In [352]:
client = GAC(key=key,b64secret=b64secret,
                 passphrase=passphrase,
                 api_url="https://api.pro.coinbase.com")
    
client.__clearingmaster__() # set clearing master
client.cm_.getorders()

In [353]:
client.cm_.heartbeat()

DAXY CANCELLED old order : ['27f67f32-7926-4626-92e6-e171a1cf26e8']
DAXY CANCELLED old order : ['96c92a01-a098-45b2-8181-38992b6db7c4']


In [345]:
t

Timestamp('2018-08-17 16:52:20.662225')

In [346]:
dfs = client.cm_.df_openorders

In [347]:
dfs[dfs['created_at']<t]

,created_at,executed_value,fill_fees,filled_size,id,post_only,price,product_id,settled,side,size,status,time_in_force,type
2,2018-08-17 15:12:27.685553,0.0000000000000000,0.0000000000000000,0.00000000,96c92a01-a098-45b2-8181-38992b6db7c4,True,3000.00000000,BTC-EUR,False,buy,0.00100000,open,GTC,limit


In [321]:
t = pd.to_datetime(time.time()-6000,unit='s')

In [332]:
client.cm_.df_openorders['']

,created_at,executed_value,fill_fees,filled_size,id,post_only,price,product_id,settled,side,size,status,time_in_force,type
0,2018-08-17 17:03:32.650350,0.0000000000000000,0.0000000000000000,0.00000000,27f67f32-7926-4626-92e6-e171a1cf26e8,False,3000.00000000,BTC-EUR,False,buy,0.01000000,open,GTC,limit
2,2018-08-17 15:12:27.685553,0.0000000000000000,0.0000000000000000,0.00000000,96c92a01-a098-45b2-8181-38992b6db7c4,True,3000.00000000,BTC-EUR,False,buy,0.00100000,open,GTC,limit


In [323]:
dfs[dfs['created_at']>t]

,created_at,executed_value,fill_fees,filled_size,id,post_only,price,product_id,settled,side,size,status,time_in_force,type
0,2018-08-17 17:03:32.650350,0.0000000000000000,0.0000000000000000,0.00000000,27f67f32-7926-4626-92e6-e171a1cf26e8,False,3000.00000000,BTC-EUR,False,buy,0.01000000,open,GTC,limit


In [312]:
dfs[(dfs['created_at'] < pd.to_datetime(time.time()-1000,unit='s'))==True]

,created_at,executed_value,fill_fees,filled_size,id,post_only,price,product_id,settled,side,size,status,time_in_force,type
0,2018-08-17 17:03:32.650350,0.0000000000000000,0.0000000000000000,0.00000000,27f67f32-7926-4626-92e6-e171a1cf26e8,False,3000.00000000,BTC-EUR,False,buy,0.01000000,open,GTC,limit
2,2018-08-17 15:12:27.685553,0.0000000000000000,0.0000000000000000,0.00000000,96c92a01-a098-45b2-8181-38992b6db7c4,True,3000.00000000,BTC-EUR,False,buy,0.00100000,open,GTC,limit


In [238]:
df_openorders = pd.DataFrame(openorders).query('product_id == "BTC-EUR"')

In [333]:
if 5 > 4 as e:
    print(e)

SyntaxError: invalid syntax (<ipython-input-333-aec5782e148d>, line 1)

In [249]:
df_openorders.loc['created_at'] = pd.to_datetime(df_openorders['created_at'])

In [250]:
df_openorders.iloc[0][0]

Timestamp('2018-08-17 17:03:32.650350')

In [202]:
self.openorders = client.get_orders()[0]
self.df_openorders = pd.DataFrame(self.openorders).query('product_id == "BTC-EUR"')
self.df_openorders.loc['created_at'] = pd.to_datetime(self.df_openorders['created_at'])
self.df_openorders = self.df_openorders.T 

,0,2
created_at,2018-08-17T17:03:32.65035Z,2018-08-17T15:12:27.685553Z
executed_value,0.0000000000000000,0.0000000000000000
fill_fees,0.0000000000000000,0.0000000000000000
filled_size,0.00000000,0.00000000
id,27f67f32-7926-4626-92e6-e171a1cf26e8,96c92a01-a098-45b2-8181-38992b6db7c4
post_only,False,True
price,3000.00000000,3000.00000000
product_id,BTC-EUR,BTC-EUR
settled,False,False
side,buy,buy


In [205]:
o.loc['created_at'][0]

'2018-08-17T17:03:32.65035Z'

In [208]:
pd.to_datetime(o.loc['created_at'])>time.time()

TypeError: invalid type comparison

In [ ]:
pd.to_datetime(o.loc{})